In [15]:
# Imports

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import tensorflow as tf
import numpy as np
import pandas as pd
from itertools import *
import sklearn
import math
import random
import sys
import multiprocessing
import scipy
from joblib import Parallel, delayed
import threading
nproc = max(1, multiprocessing.cpu_count() - 1)

if 'utils' not in sys.path:
    sys.path.append('utils')

import data_loader

# Warnings

import warnings
warnings.filterwarnings('ignore')

# Idempotent, cached data retrieval script

print(data_loader.load_chromosome.__doc__)
train_df, test_df, train_ix, test_ix, train_tissues, tfs = \
    data_loader.load_chromosome_cached('1')
    
sess = tf.InteractiveSession()

Idempotent data loading. For a given chromosome n (a string).
    
    Returns (train_df, test_df, train_ix, test_ix, train_tissues, tfs)
    
    The first two are the train and test dataframes, and test_ix are the
    values in test_df['assayed'] that are missing and need to be imputed (with the
    correct answer being in test_df['filled'] in the corresponding locations.
    train_ix are the assayed (known) methylation values from limited microarray
    sampling (e.g., test_df['assayed'].iloc[train_ix] can be used for prediction of
    test_df['filled'].iloc[test_ix], and the former should be about equal to
    test_df['filled'].iloc[train_ix] (two different ways of sampling methylation).
    
    Imports genetic context and adds those columns to the parameter df, returning
    a merged one. tfs is the list of names of new transcription
    factors.
    
    train_tissues is a list of the names of columns with chromosome methylation values.
    
    Note that loading from scratch ma

In [16]:
# Perhaps there are obvious sequence trends?
def local_impute(data):
    #http://stackoverflow.com/questions/9537543/replace-nans-in-numpy-array-with-closest-non-nan-value
    mask = np.isnan(data)
    data[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data[~mask])
    return data

# Do mean imputation on our training data.
def mean_impute(data):
    mask = np.isnan(data)
    data[mask] = float(data.mean()) # just = m messes with serialization
    return data
train_df_imp = train_df
train_df_int = train_df
for i in train_tissues:
    train_df_imp[i] = mean_impute(train_df[i].copy())
    train_df_int[i] = local_impute(train_df[i].copy())
print('nans in mean-imputed', np.isnan(train_df_imp[train_tissues]).sum().sum())
print('nans in interpolated', np.isnan(train_df_int[train_tissues]).sum().sum())

nans in mean-imputed 0
nans in interpolated 0


In [17]:
# Copied pretty much directly from sklearn

# Returns a TensorFlow scalar with the size of the i-th dimension for
# the parameter tensor x.
def tf_get_shape(x, i):
    return tf.squeeze(tf.slice(tf.shape(x), [i], [1])) 

def tf_nrows(x):
    return tf_get_shape(x, 0)

def tf_ncols(x):
    return tf_get_shape(x, 1)

# Simultaneous K-cluster likelihood computation.
# X is NxD, mus is KxD, sigmas is KxD
# Output is KxN likelihoods for each sample in each cluster.
def tf_log_normals(X, mus, sigmas):
    # p(X) = sqrt(a * b * c)
    # a = (2 pi)^(-p)
    # b = det(sigma)^(-1)
    # c = exp(-(x - mu)^T sigma^(-1) (x - mu)) [expanded for numerical stability]
    #
    # Below we make simplifications since sigma is diag
    
    D = tf_ncols(mus)
    XT = tf.transpose(X) # pxN
    invsig = tf.inv(sigmas)
    
    loga = -tf.cast(D, 'float64') * tf.log(tf.constant(2 * np.pi, dtype='float64')) # scalar
    logb = tf.reduce_sum(tf.log(invsig), 1, keep_dims=True) # Kx1
    logc =  \
        - tf.reduce_sum(invsig * tf.square(mus), 1, keep_dims=True) \
        + 2 * tf.matmul(invsig * mus, XT) \
        - tf.matmul(invsig, tf.square(XT)) # KxN
    
    return 0.5 * (loga + logb + logc)

# Stably log-sum-exps likelihood along rows.
# Reduces KxN tensor L to 1xN tensor
def tf_log_sum_exp(L):
    maxs = tf.reduce_max(L, 0, keep_dims=True) # 1xN
    return tf.log(tf.reduce_sum(tf.exp(L - maxs), 0, keep_dims=True)) + maxs

# X is NxD, mus is KxD, sigmas KxD, alphas is K
# output is KxN log likelihoods.
def tf_log_likelihood(X, mus, sigmas, alphas):
    alphas = tf.expand_dims(alphas, 1) # Kx1
    return tf_log_normals(X, mus, sigmas) + tf.log(alphas) # KxN

# X is NxD, mus is KxD, sigmas KxD, alphas is K
# output is 1xN log probability for each sample, KxN responsibilities
def estep(X, mus, sigmas, alphas):
    log_likelihoods = tf_log_likelihood(X, mus, sigmas, alphas)
    sample_log_prob = tf_log_sum_exp(log_likelihoods) # 1xN
    return sample_log_prob, tf.exp(log_likelihoods - sample_log_prob)

EPS = np.finfo(float).eps
MIN_COVAR = 0.1
# X is NxD, resp is KxN (and normalized along axis 0)
# Returns maximize step means, covariance, and cluster priors,
# which have dimension KxD, KxD, and K, respectively
def mstep(X, resp):
    weights = tf.reduce_sum(resp, 1) # K
    invweights = tf.expand_dims(tf.inv(weights + 10 * EPS), 1) # Kx1
    alphas = EPS + weights / (tf.reduce_sum(weights) + 10 * EPS) # K
    weighted_cluster_sum = tf.matmul(resp, X) # KxD 
    mus = weighted_cluster_sum * invweights
    avg_X2 = tf.matmul(resp, tf.square(X)) * invweights
    avg_mu2 = tf.square(mus)
    avg_X_mu = mus * weighted_cluster_sum * invweights
    sigmas = avg_X2 - 2 * avg_X_mu + avg_mu2 + MIN_COVAR
    # (x - mu) (x-mu)^T for banded. 
    return mus, sigmas, alphas

In [18]:
# Make sure N, D are small so the numerically unstable verification code
# doesn't underflow.
N = 5
D = 10

X = np.random.normal(size=(N, D))
mu = X.mean(axis=0)
sigma = X.std(axis=0)
mus = np.array([mu, mu, mu * 2])
sigmas = np.array([sigma, sigma * 2, sigma])
K = len(sigmas)
alphas = np.random.dirichlet(np.ones(K), 1)[0]

mean_ll, resp = sess.run(estep(*(tf.constant(x) for x in (X, mus, sigmas, alphas))))
def normal_likelihoods(X, mu, sigma):
    exponent = -np.dot((X - mu[np.newaxis, :]) ** 2, 1 / sigma) / 2
    return (2 * np.pi) ** (-D / 2) * np.prod(sigma) ** (-1 / 2) * np.exp(exponent)
actual = np.array([normal_likelihoods(X, mu, sigma) for mu, sigma in zip(mus, sigmas)])
actual = sklearn.preprocessing.normalize(actual * alphas[:, np.newaxis], norm='l1', axis=0)
resp = sklearn.preprocessing.normalize(resp, norm='l1', axis=0)
print('actual likelihoods', actual)
print('log likelihoods   ', resp)
rmses = np.sqrt(sklearn.metrics.mean_squared_error(actual.T, resp.T, multioutput='raw_values'))
for i, rmse in enumerate(rmses):
    print('K={} rmse={}'.format(i, rmse))

actual likelihoods [[ 0.26088298  0.29452575  0.0762016   0.30748466  0.45977218]
 [ 0.39989859  0.22567318  0.37302021  0.3812018   0.1812242 ]
 [ 0.33921843  0.47980106  0.5507782   0.31131355  0.35900362]]
log likelihoods    [[ 0.26088298  0.29452575  0.0762016   0.30748466  0.45977218]
 [ 0.39989859  0.22567318  0.37302021  0.3812018   0.1812242 ]
 [ 0.33921843  0.47980106  0.5507782   0.31131355  0.35900362]]
K=0 rmse=3.2676223177844214e-16
K=1 rmse=2.9634845277824204e-16
K=2 rmse=3.466670283279959e-16


In [19]:
# Similar pattern to
# https://gist.github.com/narphorium/d06b7ed234287e319f18

#todo try initializing covar to emprical cv computed from kmeans labels

# Runs up to max_steps EM iterations, stopping earlier if log likelihood improves
# less than tol.
# X should be an NxD data matrix, initial_mus should be KxD
# max_steps should be an int, tol should be a float.
def fit_em(X, initial_mus, max_steps, tol, sess):
    N, D = X.shape
    K, Dmu = initial_mus.shape
    assert D == Dmu
        
    mus0 = initial_mus
    sigmas0 = np.tile(np.var(X, axis=0), (K, 1))
    alphas0 = np.ones(K) / K
    X = tf.constant(X)
    
    mus, sigmas, alphas = (tf.Variable(x, dtype='float64') for x in [mus0, sigmas0, alphas0])
    
    sess.run(tf.initialize_all_variables())
    all_ll, resp = estep(X, mus, sigmas, alphas)
    cmus, csigmas, calphas = mstep(X, resp)
    update_mus_step = tf.assign(mus, cmus)
    update_sigmas_step = tf.assign(sigmas, csigmas)
    update_alphas_step = tf.assign(alphas, calphas)     
    
    init_op = tf.initialize_all_variables()
    ll = prev_ll = -np.inf
                         
    with tf.Session() as sess2:
        sess2.run(init_op)
        for i in range(max_steps):
            ll = sess2.run(tf.reduce_mean(all_ll))
            sess2.run((update_mus_step, update_sigmas_step, update_alphas_step))
            print('EM iteration', i, 'log likelihood', ll)
            if abs(ll - prev_ll) < tol:
                break
            prev_ll = ll
        m, s, a = sess2.run((mus, sigmas, alphas))
    
    return ll, m, s, a

In [20]:
# Given a set of partial observations xs each of dimension O < D for a fitted GMM model with 
# K cluster priors alpha, KxD means mus, and KxD diagonal covariances sigmas,
# returns the weighted sum of normals for the remaining D - O dimensions.
#
# Returns posterior_mus, posterior_sigmas, posterior_prior,
# of dimensions:
# Kx(D-O), Kx(D-O), NxK, respectively (each mu, sigma is the same for all posteriors).
# NxK, NxKxD, NxKxD, respectively, for each x in xs, total of N.
def marginal_posterior(xs, mus, sigmas, alphas, sess):
    # https://gbhqed.wordpress.com/2010/02/21/conditional-and-marginal-distributions-of-a-multivariate-gaussian/
    # diagonal case is easy:
    # https://en.wikipedia.org/wiki/Schur_complement#Applications_to_probability_theory_and_statistics
    O = xs.shape[1]
    D = mus.shape[1]
    observed_mus, observed_sigmas = (tf.constant(a, dtype='float64')
                                     for a in (mus[:,0:O], sigmas[:, 0:O]))
    ll = tf_log_likelihood(xs, observed_mus, observed_sigmas, alphas) # KxN
    norm = tf_log_sum_exp(ll) # 1xN
    ll, norm = sess.run((ll, norm))
    return mus[:, O:D], sigmas[:, O:D], np.transpose(ll - norm)

# A "sparser" estimate which just uses the most likely cluster's mean as the estimate.
def argmax_exp(mus, sigmas, alphas):
    i = np.argmax(alphas)
    return mus[i]

In [7]:
n_samples = 25
np.random.seed(0)

def MVN(shear, shift):
    rs = np.random.randn(n_samples, 2)
    return np.dot(rs, shear.T) + shift
G1 = MVN(np.identity(2), np.array([20, 20]))
G2 = MVN(np.array([[0., 3.5], [-0.7, .7]]), np.zeros(2))
G3 = MVN(np.array([[-1, 0.8], [0, 4]]), np.array([10, 8]))

# concatenate the two datasets into the final training set
X = np.vstack([G1, G2, G3])
rx = np.random.choice(range(len(X)), 3, replace=False)

# http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html
_, m, s, a = fit_em(X, X[rx], 100, EPS)
x = np.linspace(-20.0, 30.0)
y = np.linspace(-20.0, 40.0)
xx, yy = np.meshgrid(x, y)
pts = np.array([xx.ravel(), yy.ravel()]).T
ll = -estep(pts, m, s, a)[0].eval()
print('means\n{}\ncov\n{}\ncluster priors\n{}'.format(m, s, a))
CS = plt.contour(xx, yy, ll.reshape(xx.shape), norm=LogNorm(vmin=1.0, vmax=1000.0),
                levels=np.logspace(0, 3, 20))
plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X[:, 0], X[:, 1])
plt.show()

TypeError: fit_em() missing 1 required positional argument: 'sess'

In [8]:
# Make this into a nice image.
ys = [-10, 0, 2.5, 10, 20]
pts = np.arange(-20, 30, 0.1, dtype='float64').reshape(-1, 1)
ys = np.array(ys, dtype='float64').reshape(-1, 1)
# Reverse m, s, a, since we know 'y'
mr, sr = (x[:, ::-1] for x in (m, s))
mm, sm, ams = marginal_posterior(ys, mr, sr, a)
for i, y in enumerate(ys.reshape(-1)):
    print('y =', y)
    ll = estep(pts, mm, sm, ams[i])[0].eval().reshape(-1)
    plt.plot(pts, np.exp(ll) / 10 + y)
    #plt.show()

ys = np.arange(-20, 30, 0.05, dtype='float64').reshape(-1, 1)
mm, sm, ams = marginal_posterior(ys, mr, sr, a)
xs = [argmax_exp(mm, sm, am) for am in ams]
plt.plot(xs, ys, label='argmax cluster mu')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

NameError: name 'm' is not defined

In [9]:
# N = 10
N = 33
p = 20 # D is better?
K = 6
k = 0 # maximum band offset (0 is just diagonal)
tissues_to_cluster = train_tissues
X_np = train_df_imp[train_tissues].values.transpose()

np.random.seed(2)
rc = np.random.choice(range(len(train_df)), D, replace=False)
rmu = np.random.choice(range(N), K, replace=False)

np.random.shuffle(X_np)
per = N // K
splits = [X_np[i:i+per] for i in range(0, N // K * K, per)]


# X_trunc = X_np[:N, rc]
X_trunc = X_np[:N]
mu_init = X_trunc[rmu]
# mu_init = np.array([x.mean(axis=0) for x in splits])
#mu_init = X_np[19:22]


print(mu_init.shape)

(6, 379551)


In [10]:
print(X_np.shape)



(34, 379551)


In [42]:
# Fix a value of K. Perform NUM_RESTARTS random restarts, and pick EM fit with highest mean likelihood 
# X_np_new = data matrix with bad samples deleted.
# X_perm   = permute(X_np_new)
# K        = number of clusters
# NUM_RESTARTS = number of random restarts

K = 5
NUM_RESTARTS = 1

def get_permutation():
    o = np.ones(len(train_df))
    o[train_ix] = 0
    o[test_ix] = 0
    unobserved_untested_ix = np.where(o)[0]
    o = np.zeros(len(train_df))
    o[test_ix] = 1
    unobserved_tested_ix = np.where(o)[0]
    permutation = np.hstack((train_ix, unobserved_tested_ix, unobserved_untested_ix))
    return permutation, unobserved_tested_ix, unobserved_untested_ix

def fit_model(X_train, unobserved_tested_ix, unobserved_untested_ix):
    tf.reset_default_graph()
    lp = None
    m = None
    s = None
    a = None
    for j in range(NUM_RESTARTS):
        rmu = np.random.choice(range(X_train.shape[0]), K, replace=False)
        mu_init = X_train[rmu]
        cur_lp, cur_m, cur_s, cur_a = fit_em(X_train, mu_init, 100, EPS, tf.Session())
        if lp is None or cur_lp > lp:
            lp = cur_lp
            m = cur_m
            s = cur_s
            a = cur_a
    print("Done picking best EM")
    
    # X is NxD, mus is KxD, sigmas KxD, alphas is K
    # output is 1xN log probability for each sample, KxN responsibilities

    observed = test_df['filled'][train_ix].values
    marginal_means, marginal_covs, marginal_alphas = marginal_posterior(observed.reshape(1, len(observed)), m, s, a, tf.Session())

    pred = argmax_exp(marginal_means, marginal_covs, marginal_alphas[0])[:len(unobserved_tested_ix)]
    actual = test_df['filled'][unobserved_tested_ix]
    print(len(pred), len(actual))
    rmse = sklearn.metrics.mean_squared_error(actual, pred)
    print('rmse', np.sqrt(rmse)) # rmse of GMM
    r2 = sklearn.metrics.r2_score(actual, pred)
    print('r2', r2)
    return marginal_means, marginal_covs, marginal_alphas, lp, m, s, a
    
    
'''
    observed = X_test[:len(train_ix)]
    marginal_means, marginal_covs, marginal_alphas = marginal_posterior(observed.reshape(1, len(observed)), m, s, a, sess)

    pred = argmax_exp(marginal_means, marginal_covs, marginal_alphas[0])[:len(unobserved_tested_ix)]
    actual = X_test[len(train_ix):len(train_ix)+len(unobserved_tested_ix)]
    print(len(pred), len(actual))
    
    # Compute the rmse and r2
    mse = sklearn.metrics.mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    print('rmse', rmse) # rmse of GMM
    r2 = sklearn.metrics.r2_score(actual, pred)
    print('r2', r2)
'''

print("X_np.shape", X_np.shape) 

# delete bad samples
X_np_new = X_np[:][:33]
X_np_new = np.delete(X_np_new, [14,25,26], 0)
print(X_np_new.shape)

perm, unobserved_tested_ix, unobserved_untested_ix = get_permutation()
print(perm)
X_perm = X_np_new[:, perm]

print("K", K)
print("NUM_RESTARTS", NUM_RESTARTS)
print("X_perm.shape", X_perm.shape) 

#KxN responsibilities
marginal_means, marginal_covs, marginal_alphas, marginal_logp, mus, sigs, alphas = fit_model(X_perm, unobserved_tested_ix, unobserved_untested_ix)
print(marginal_alphas)

X_np.shape (34, 379551)
(30, 379551)
[  1084   1154   1214 ..., 378792 378806 379168]
K 5
NUM_RESTARTS 1
X_perm.shape (30, 379551)
EM iteration 0 log likelihood 104834.753505
EM iteration 1 log likelihood 31736.3861947
EM iteration 2 log likelihood 51102.112022
EM iteration 3 log likelihood 52912.2714273
EM iteration 4 log likelihood 53244.2041499
EM iteration 5 log likelihood 53719.200865
EM iteration 6 log likelihood 53719.200865
Done picking best EM
368411 368411
rmse 0.0698955993458
r2 0.829151582451
[[-1691.52418439     0.         -1930.65356754 -1114.06471985
  -4064.88742012]]


In [68]:
print(marginal_alphas.shape)
print(marginal_covs.shape)
print(marginal_means.shape)

K = marginal_alphas.shape[1]

print("before:", np.sum(marginal_alphas))
alphs = np.exp(marginal_alphas)
print("after:", np.sum(alphs))
alphs = np.ndarray.tolist(alphs[0,:])

SAMPLE_SIZE = 100

# Pick a random clusters
samples = np.random.choice(len(alphas), SAMPLE_SIZE, p=alphas)
cnts = np.bincount(samples)
print(cnts)

print(marginal_means[1,:])

#S = [] #np.zeros((SAMPLE_SIZE, 2))
#for s in samples:
    # Pick a sample from this normal
    #np.random.normal(marginal_means[s,:], marginal_covs[s], size=(K,N))
    
    # Add the sample to the set of samples

(1, 5)
(5, 372028)
(5, 372028)
before: -8801.12989189
after: 1.0
[14 39 41  4  2]
[ 0.91995043  0.93998369  0.92546759 ...,  0.19855093  0.76246235
  0.91682623]
